In [107]:
# Librerias
import math
import pandas as pd
import numpy as np

Para escoger el mejor camino de BOA, o NGSA-II , se deben tener guardado los caminos del algoritmo en un excel cuyas columnas sean 'N. solucion', 'Distancia' y 'Contaminacion'.

Para obtener el ranking de algoritmos se debe tener guardado los resultados de los algoritmos en un excel, cuyas columnas sean 'Algoritmo', 'Distancia', 'Contaminacion' y 'Tiempo de ejecucion'.

In [108]:
file_path = "resultados.xlsx"
df = pd.read_excel(file_path)

In [109]:
df

,Algoritmo,Distancia (m),Contaminacion (%),Tiempo de procesamiento (seg)
0,Dijkstra min dist,7.230581,94.052809,0.263563
1,Dijkstra min polucion,12.230042,31.886277,0.204578
2,Astar min dist,7.230581,94.052809,0.297704
3,Astar min polucion,12.230042,31.886277,0.230880
4,D-e,9.701702,49.502686,0.224505
5,A-e,9.701702,49.502686,0.257728
6,Pulse,7.863335,49.898616,23.354958
7,Boa,8.577492,42.786975,8.923044


In [110]:
#alternatives = df['N. solucion']
#criteria = df.drop('N. solucion', axis=1)
alternatives = df['Algoritmo']
criteria = df.drop('Algoritmo', axis=1)
criteria_matrix = criteria.values

In [111]:
m = criteria_matrix.shape[1]

# SHANNON

In [112]:
normal_criteria_matrix = np.zeros_like(criteria_matrix, dtype=float)

for col_idx in range(criteria_matrix.shape[1]):
    col_sum = np.sum(criteria_matrix[:, col_idx])
    normal_criteria_matrix[:, col_idx] = criteria_matrix[:, col_idx] / col_sum

In [113]:
k = 1/math.log(m)

In [114]:
normal_ln_criteria_matrix = np.log(normal_criteria_matrix)

In [115]:
ej = -k * np.sum((normal_criteria_matrix * normal_ln_criteria_matrix), axis=0)

In [116]:
wj = ((1-ej)/np.sum(ej))*(1-ej)

In [117]:
df_wj = pd.DataFrame({'Criteria': criteria.columns.tolist(), 'Wj': wj})

In [118]:
df_wj_sorted = df_wj.sort_values(by='Wj', ascending=False)

## Pesos por SHANNON

In [119]:
print(df_wj_sorted.to_string(header=False, index=False))

                Distancia (m) 0.172237
            Contaminacion (%) 0.150616
Tiempo de procesamiento (seg) 0.014300


# SHANNON-TOPSIS

In [120]:
ideal_criteria = np.max(df.iloc[:, 1:].to_numpy(), axis=0)
antideal_criteria = np.min(df.iloc[:, 1:].to_numpy(), axis=0)

In [121]:
ideal_criteria

array([12.23004231, 94.05280944, 23.35495758])

In [122]:
antideal_criteria

array([ 7.23058136, 31.88627667,  0.20457768])

In [123]:
norm_criteria = np.linalg.norm(df.iloc[:, 1:], axis=0)

In [124]:
norm_criteria

array([ 26.9696485 , 170.13806242,  25.00889528])

In [125]:
topsis_criteria_norm_matrix = criteria.div(norm_criteria)

In [126]:
topsis_criteria_norm_matrix

,Distancia (m),Contaminacion (%),Tiempo de procesamiento (seg)
0,0.268101,0.552803,0.010539
1,0.453474,0.187414,0.008180
2,0.268101,0.552803,0.011904
3,0.453474,0.187414,0.009232
4,0.359727,0.290956,0.008977
5,0.359727,0.290956,0.010305
6,0.291562,0.293283,0.933866
7,0.318042,0.251484,0.356795


In [127]:
ideal_topsis_criteria = np.max(topsis_criteria_norm_matrix.to_numpy(), axis=0)
antideal_topsis_criteria = np.min(topsis_criteria_norm_matrix.to_numpy(), axis=0)

In [128]:
ideal_topsis_criteria

array([0.45347429, 0.55280287, 0.93386602])

In [129]:
antideal_topsis_criteria

array([0.26810069, 0.18741413, 0.0081802 ])

In [130]:
ideal_shannon = ideal_topsis_criteria*wj

In [131]:
ideal_shannon

array([0.07810527, 0.08326095, 0.01335415])

In [132]:
antideal_shannon = antideal_topsis_criteria*wj

In [133]:
antideal_shannon

array([0.04617699, 0.02822756, 0.00011698])

In [134]:
pon_shannon =  topsis_criteria_norm_matrix.mul(wj)

In [135]:
pon_shannon

,Distancia (m),Contaminacion (%),Tiempo de procesamiento (seg)
0,0.046177,0.083261,0.000151
1,0.078105,0.028228,0.000117
2,0.046177,0.083261,0.000170
3,0.078105,0.028228,0.000132
4,0.061958,0.043823,0.000128
5,0.061958,0.043823,0.000147
6,0.050218,0.044173,0.013354
7,0.054779,0.037877,0.005102


In [136]:
ideal_wj = (pon_shannon.sub(ideal_shannon)) ** 2

In [137]:
euclidean_norms_ideal_wj = np.linalg.norm(ideal_wj.values, axis=1)

In [138]:
antideal_wj = (pon_shannon.sub(antideal_shannon)) ** 2

In [139]:
euclidean_norms_antideal_wj = np.linalg.norm(antideal_wj.values, axis=1)

In [140]:
rsi = euclidean_norms_antideal_wj/(euclidean_norms_ideal_wj+euclidean_norms_antideal_wj)

In [141]:
rsi

array([0.74544852, 0.25151165, 0.74546442, 0.25151307, 0.17991223,
       0.17991737, 0.1528061 , 0.05393312])

In [142]:
sorted_algorithms = np.argsort(rsi)

In [143]:
sorted_algorithms

array([7, 6, 4, 5, 1, 3, 0, 2], dtype=int64)

In [144]:
sorted_algorithms = sorted_algorithms.tolist()

# Clasificacion de algoritmos

In [145]:
for i in range(0, len(sorted_algorithms)):
  print(alternatives[sorted_algorithms[i]])

Boa
Pulse
D-e
A-e
Dijkstra min polucion
Astar min polucion
Dijkstra min dist
Astar min dist
